In [282]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [283]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.royallepage.ca/en/search/homes/?search_str=Ontario%2C+ON%2C+CAN&csrfmiddlewaretoken=JFtNsGhr8TWywqOO8l6zJx2pj75oWqsga7nw8m1ggOUv1KxpzmwBLYmerFUD6x7f&property_type=7%2C8&house_type=102%2C103%2C104&features=&listing_type=&lat=50.926163435000035&lng=-84.74492999999995&upper_lat=&upper_lng=&lower_lat=&lower_lng=&bypass=&radius=&zoom=&display_type=gallery-view&travel_time=&travel_time_min=30&travel_time_mode=drive&travel_time_congestion=&da_id=&segment_id=&tier2=False&tier2_proximity=0&address=Ontario&method=homes&address_type=province&city_name=&prov_code=ON&school_id=&min_price=0&max_price=3000000&min_leaseprice=0&max_leaseprice=5000%2B&beds=0&baths=0&transactionType=SALE&archive_status=All&archive_timespan=6&sfproperty_type%5B7%5D=7&sfproperty_type%5B8%5D=8&sfhouse_type%5B102%5D=102&sfhouse_type%5B103%5D=103&sfhouse_type%5B104%5D=104&keyword=&sortby="

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [284]:
# Go through all pages to collect data
house_listings = []
for x in range(1, 23):
    html = browser.html
    soup = bs(html, 'html.parser')
    links = soup.find_all(class_='address-1')
    
    # Get all address listings for buying    
    print('Gathering data from page:', x)
    for link in links:
        house_listings.append(link.a['href'])
    browser.find_by_css('.next').click()

# Number of listings
len(house_listings)


Gathering data from page: 1
Gathering data from page: 2
Gathering data from page: 3
Gathering data from page: 4
Gathering data from page: 5
Gathering data from page: 6
Gathering data from page: 7
Gathering data from page: 8
Gathering data from page: 9
Gathering data from page: 10
Gathering data from page: 11
Gathering data from page: 12
Gathering data from page: 13
Gathering data from page: 14
Gathering data from page: 15
Gathering data from page: 16
Gathering data from page: 17
Gathering data from page: 18
Gathering data from page: 19
Gathering data from page: 20
Gathering data from page: 21
Gathering data from page: 22


1012

In [285]:
browser.quit()

In [286]:
# Dependencies
import requests
import json

# Import the API key
from config import apiKey

# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)


# Visit each listing and grab relevant data
house_listing_info = []
for i in range(len(house_listings)):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 
    
    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    
    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text


    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    values = building_feature_section.find_all('span', class_="value")
    
# Feastures of house
    style_type = values[0].text.strip(":")
    cooling_type = values[-1].text
    heating_fuel = values[-2].text
    heating_type = values[-3].text
    fire_place = values[-4].text
    houses['address'] = address
    houses['price'] = price
    houses['number_of_beds'] = bed_count
    houses['number_of_baths'] = bath_count
    houses['style'] = style_type
    houses['cooling_type'] = cooling_type
    houses['heating_fuel'] = heating_fuel
    houses['heating_type'] = heating_type
    houses['fire_place'] = fire_place
    
    # Build target url with address and apikey
    string_search = houses['address'].split(",")[1:]
    new_string = (",").join(string_search).strip()                           
    
    target_url = f"https://api.geoapify.com/v1/geocode/search?text={new_string}&format=json&apiKey={apiKey}"
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()

    # Get coordinate information
    lat = geo_data["results"][0]["lat"]
    lon = geo_data["results"][0]["lon"]
    
    houses['lat'] = lat
    houses['lon'] = lon
        
    print(f"Collected data for: {houses['address']}")
    house_listing_info.append(houses)

browser.quit()


Collected data for: 121 WATERVIEW COMMON, Oakville, Ontario, L6L 0E7
Collected data for: 1041 BRANDYWINE COURT, Ottawa, Ontario, K4M 1J2
Collected data for: 24 LOVE CRT, Richmond Hill, Ontario, L4B 0G2
Collected data for: 10 CHESTER HILL RD, Toronto, Ontario, M4K 1X3
Collected data for: 88 FARRELL RD, Vaughan, Ontario, L6A 4W7
Collected data for: 177623 GREY ROAD 18, Georgian Bluffs, Ontario, N4K 5N5
Collected data for: 6415 DONWAY DR, Mississauga, Ontario, L5V 1J4
Collected data for: 296 INDIAN GRVE, Toronto, Ontario, M6P 2H5
Collected data for: 21 FIRE ROUTE 28, North Kawartha Twp, Ontario, K0L 2H0
Collected data for: 1100 LAURIER AVENUE, LaSalle, Ontario, N9J 1N3
Collected data for: 1001 Chateau, Windsor, Ontario, N8P 0E6
Collected data for: 12 SOMERSET AVE, Toronto, Ontario, M6H 2R4
Collected data for: 36 BROOKLAND AVE, Aurora, Ontario, L4G 2H8
Collected data for: 15 KEYSTONE Crescent, Hamilton, Ontario, L0R 1P0
Collected data for: 234 TAMPICO RD, Richmond Hill, Ontario, L4C 4R7
Co

IndexError: list index out of range

In [ ]:
#  # House features
# building_feature_section = soup.find('ul', class_="property-features-list")
# labels = building_feature_section.find_all('span', class_="label")
# values = building_feature_section.find_all('span', class_="value")

# wanted_feature = ['Fireplace', 'Style', 'Heating Type', 'Heating Fuel', 'Cooling Type']
# listed_features = [label.text.strip(":") for label in labels]
# value_features = [value.text.strip() for value in values]

# for i in range(len(wanted_feature)):
#     if (wanted_feature[i] in listed_features):
#         print("Yes")
#     else:
#         print("No")

# # [i in listed_features for i in wanted_feature]:
    
# print(style_type)
 


No
Yes
Yes
Yes
Yes
0


In [287]:
len(house_listing_info)

106

In [ ]:
house_listing_info

[{'address': '2183 PHEASANT LANE, Oakville, Ontario, L6M 3R8',
  'price': '$1,998,000',
  'number_of_beds': '4+1 Beds',
  'number_of_baths': '4 Baths',
  'style': 'Detached',
  'cooling_type': 'Central air conditioning',
  'heating_fuel': 'Natural gas',
  'heating_type': 'Forced air',
  'fire_place': 'Yes',
  'lat': 43.447436,
  'lon': -79.666672},
 {'address': '32 ATLANTIC AVE, Markham, Ontario, L3P 7C4',
  'price': '$1,899,900',
  'number_of_beds': '4+2 Beds',
  'number_of_baths': '4 Baths',
  'style': 'Detached',
  'cooling_type': 'Central air conditioning',
  'heating_fuel': 'Natural gas',
  'heating_type': 'Forced air',
  'fire_place': 'Yes',
  'lat': 43.8563707,
  'lon': -79.3376825},
 {'address': '27 WATTS MEADOW, Aurora, Ontario, L4G 7L7',
  'price': '$1,898,888',
  'number_of_beds': '4 Beds',
  'number_of_baths': '5 Baths',
  'style': 'Detached',
  'cooling_type': 'Central air conditioning',
  'heating_fuel': 'Natural gas',
  'heating_type': 'Forced air',
  'fire_place': 'Yes'

In [ ]:
import re

txt = '2183 PHEASANT LANE, Oakville, Ontario, L6M 3R8'

search_string = re.sub('.+([A-Z]|[0-9])\s[-]','', txt)       
search_string                  
                           

'2183 PHEASANT LANE, Oakville, Ontario, L6M 3R8'

In [ ]:
import csv

try:
    with open('./Resources/housing.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['address', 'price', 'number_of_beds', 'number_of_baths', 'style', 'cooling_type','heating_fuel', 'heating_type', 'fire_place', 'lat', 'lon'])
        writer.writeheader()
        for data in house_listing_info:
            writer.writerow(data)
except IOError:
    print("I/O error")